In [266]:
import nltk
import os

In [267]:
#C(wi, ti)
word_tag_counts = {}

#C(ti)
tag_unigram_counts = {}

# C(ti−1, ti)
tag_bigram_counts = {}


files = os.listdir('brown_train')
for file in files:

    text = open('brown_train/' + file).read()
    text = text.replace('\t', '')
    sentences = text.splitlines()
    for sent in sentences:
        if sent == '':
            continue
        
        word_tag = sent.split()
        word_tag.insert(0, '/start')
        word_tag.append('/end')
        for i in range(0, len(word_tag)):
            word, tag = word_tag[i].rsplit('/', 1)

            try:
                word_tag_counts[(word,tag)] += 1
            except:
                word_tag_counts[(word,tag)] = 1

            try:
                tag_unigram_counts[tag] += 1
            except:
                tag_unigram_counts[tag] = 1

            if i > 0:
                try:
                    tag_bigram_counts[previous_tag, tag] += 1
                except:
                    tag_bigram_counts[previous_tag, tag] = 1
            previous_tag = tag;

    

In [88]:
# write to file
with open('word_tag_counts.txt','w') as f:
    f.write(str(word_tag_counts))
with open('tag_unigram_counts.txt','w') as f:
    f.write(str(tag_unigram_counts))
with open('tag_bigram_counts.txt','w') as f:
    f.write(str(tag_bigram_counts))

In [268]:
#P(ti−1, ti) = C(ti−1, ti) / C(ti−1)
def cal_transition_prob(tag1, tag2):
    try:
        return tag_bigram_counts[(tag1, tag2)] / tag_unigram_counts[tag1]
    except:
        return 0
    
transition_prob = {}

files = os.listdir('brown_train')
for file in files:

    text = open('brown_train/' + file).read()
    text = text.replace('\t', '')
    sentences = text.splitlines()
    for sent in sentences:
        if sent == '':
            continue
        
        word_tag = sent.split()
        word_tag.insert(0, '/start')
        word_tag.append('/end')
        for i in range(0, len(word_tag)):
            word, tag = word_tag[i].rsplit('/', 1)
            
            if i > 0:
                prob = cal_transition_prob(previous_tag, tag)
                transition_prob[previous_tag, tag] = prob
               
            previous_tag = tag;


In [269]:
#P(wi, ti) = C(wi, ti) / C(ti)

def cal_emission_probability(word, tag):
    try:
        return word_tag_counts[(word, tag)] / tag_unigram_counts[tag]
    except:
        return 0
    
emission_prob = {}

files = os.listdir('brown_train')
for file in files:

    text = open('brown_train/' + file).read()
    text = text.replace('\t', '')
    sentences = text.splitlines()
    for sent in sentences:
        if sent == '':
            continue
        
        word_tag = sent.split()
        word_tag.insert(0, '/start')
        word_tag.append('/end')
        for i in range(0, len(word_tag)):
            word, tag = word_tag[i].rsplit('/', 1)
            
            prob = cal_emission_probability(word, tag)
            emission_prob[word, tag] = prob


In [117]:
import random
def generate_sentence():
    sentence = ""
    probabilities = 1.0
    
    previous_tag = 'start'
    while(True):
        tags_choice = []
        tag_probabilities = []
        for tags, prob in transition_prob.items():
            if tags[0] == previous_tag:
                tags_choice.append(tags[1])
                tag_probabilities.append(prob)
                
        tag_draw = random.choices(tags_choice, weights = tag_probabilities, k = 1)[0]
        probabilities *= tag_probabilities[tags_choice.index(tag_draw)]
        previous_tag = tag_draw
        
        if tag_draw == 'end':
            break

        words = []
        word_probabilities = []
        for wordtag, count in emission_prob.items():
            if wordtag[1] == tag_draw:
                words.append(wordtag[0])
                word_probabilities.append(count)
        generated_word = random.choices(words, weights = word_probabilities, k = 1)[0]
        sentence += generated_word + " "
        
        probabilities *= word_probabilities[words.index(generated_word)]

    return sentence, probabilities

In [134]:
f = open(" random_sentences.txt", "w")
for i in range(0, 3):
    sent, prob = generate_sentence()
    f.write("sentence: " + sent + ", probability: " + str(prob) + "\n\n")
f.close()

In [288]:
#how to deal with unknown words????
#unknown word which tag? transition prob is 0

def Viterbi(sentence):
    # what is the list of all tags?
    distinct_tags = list(tag_unigram_counts.keys())

    #sentence = ["I", "saw", "her", "duck" ]
    sentlen = len(sentence)

    # viterbi:
    # for each step i in 1 .. sentlen,
    # store a dictionary
    # that maps each tag X
    # to the probability of the best tag sequence of length i that ends in X
    viterbi = [ ]

    # backpointer:
    # for each step i in 1..sentlen,
    # store a dictionary
    # that maps each tag X
    # to the previous tag in the best tag sequence of length i that ends in X
    backpointer = [ ]

    first_viterbi = { }
    first_backpointer = { }
    for tag in distinct_tags:
        # don't record anything for the START tag
        if tag == "start": 
            continue
        first_viterbi[ tag ] = cal_transition_prob('start', tag) * cal_emission_probability(sentence[0], tag) #cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
        first_backpointer[ tag ] = "start"

    #print(first_viterbi)
    #print(first_backpointer)

    viterbi.append(first_viterbi)
    backpointer.append(first_backpointer)

    currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])
    print( "Word", "'" + sentence[0] + "'", "current best two-tag sequence:", first_backpointer[ currbest], currbest)
    # print( "Word", "'" + sentence[0] + "'", "current best tag:", currbest)

    for wordindex in range(1, len(sentence)):
        this_viterbi = { }
        this_backpointer = { }
        prev_viterbi = viterbi[-1]

        for tag in distinct_tags:
            # don't record anything for the START tag
            if tag == "start": continue

            # if this tag is X and the current word is w, then 
            # find the previous tag Y such that
            # the best tag sequence that ends in X
            # actually ends in Y X
            # that is, the Y that maximizes
            # prev_viterbi[ Y ] * P(X | Y) * P( w | X)
            # The following command has the same notation
            # that you saw in the sorted() command.

            best_previous = ""
            best_prob = 0.0

            for prevtag in distinct_tags:

                prob = prev_viterbi.get(prevtag, 0) * cal_transition_prob(prevtag, tag) * cal_emission_probability(sentence[wordindex], tag)
                if prob >= best_prob:
                    best_previous = prevtag
                    best_prob = prob

            this_viterbi[ tag ] = prev_viterbi[ best_previous] * cal_transition_prob(best_previous, tag) * cal_emission_probability(sentence[wordindex], tag)

            this_backpointer[ tag ] = best_previous

        currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
        print( "Word", "'" + sentence[ wordindex] + "'", "current best two-tag sequence:", this_backpointer[ currbest], currbest)
        # print( "Word", "'" + sentence[ wordindex] + "'", "current best tag:", currbest)


        # done with all tags in this iteration
        # so store the current viterbi step
        viterbi.append(this_viterbi)
        backpointer.append(this_backpointer)


    # done with all words in the sentence.
    # now find the probability of each tag
    # to have "END" as the next tag,
    # and use that to find the overall best sequence
    prev_viterbi = viterbi[-1]
    best_previous = max(prev_viterbi.keys(),
                        key = lambda prevtag: prev_viterbi[ prevtag ] * cal_transition_prob(prevtag, 'end'))

    prob_tagsequence = prev_viterbi[ best_previous ] * cal_transition_prob(best_previous, 'end')

    # best tagsequence: we store this in reverse for now, will invert later
    best_tagsequence = [ "end", best_previous ]
    # invert the list of backpointers
    backpointer.reverse()

    # go backwards through the list of backpointers
    # (or in this case forward, because we have inverter the backpointer list)
    # in each case:
    # the following best tag is the one listed under
    # the backpointer for the current best tag
    current_best_tag = best_previous
    for bp in backpointer:
        best_tagsequence.append(bp[current_best_tag])
        current_best_tag = bp[current_best_tag]

    best_tagsequence.reverse()
    print( "The sentence was:", end = " ")
    for w in sentence: print( w, end = " ")
    print("\n")
    print( "The best tag sequence is:", end = " ")
    for t in best_tagsequence: print (t, end = " ")
    print("\n")
    print( "The probability of the best tag sequence is:", prob_tagsequence)
    return best_tagsequence

In [289]:
Viterbi(['He', 'goes', 'to','kjhkjh', 'school'])

Word 'He' current best two-tag sequence: start pps
('goes', 'vbz')
('to', 'in')
('to', 'in-tl')
('to', 'to')
('to', 'ql')
('to', 'nps')
('to', 'in-hl')
('to', 'to-hl')
('to', 'in-nc')
('to', 'nil')
('to', 'to-nc')
('school', 'nn-tl')
('school', 'nn')
('school', 'vb')
('school', 'nn-hl')


KeyError: 'fw-uh-tl'

In [263]:
word_tag_counts.get(('to',  'fw-uh-tl')) is None

True

In [236]:
result = []
sentence = []
with open('Test_File.txt', 'r') as f:
    for line in f:
        line = line.replace('\n', '')
        if line.startswith('< EOS >'):
            result.append(sentence)
            sentence = []
        else:
            sentence.append(line)
f = open("Viterbi algorithm.txt", "w")
for sentence in result:
    f.write(sentence[0] + "\n")
    
    sentence = sentence[1:]
    tags = Viterbi(sentence)[1:-1]
    for i in range(0, len(sentence)):
        f.write(sentence[i] + "/" + tags[i] + "\n")
    f.write('< EOS >' + "\n")
    
f.close()
    

Word 'The' current best two-tag sequence: start at
Word 'aged' current best two-tag sequence: at vbn
Word 'care' current best two-tag sequence: vbn nn
Word 'plan' current best two-tag sequence: nn nn
Word 'carries' current best two-tag sequence: nn vbz
Word 'these' current best two-tag sequence: vbz dts
Word 'benefits' current best two-tag sequence: dts nns
Word 'for' current best two-tag sequence: nns in
Word 'persons' current best two-tag sequence: in nns
Word 'over' current best two-tag sequence: nns in
Word '65' current best two-tag sequence: in cd
Word 'who' current best two-tag sequence: cd wps
Word 'are' current best two-tag sequence: wps ber
Word 'under' current best two-tag sequence: ber in
Word 'the' current best two-tag sequence: in at
Word 'social' current best two-tag sequence: at jj
Word 'security' current best two-tag sequence: jj nn
Word 'and' current best two-tag sequence: nn cc
Word 'railroad' current best two-tag sequence: cc nn
Word 'retirement' current best two-tag

Word 'in' current best two-tag sequence: nns in
Word '1953' current best two-tag sequence: in cd
Word 'and' current best two-tag sequence: cd cc
Word 'President' current best two-tag sequence: cc nn-tl
Word 'Kennedy's' current best two-tag sequence: nn-tl np$
Word 'in' current best two-tag sequence: np$ in
Word '1961' current best two-tag sequence: in cd
Word '.' current best two-tag sequence: cd .
The sentence was: The rule was enforced by demand of Sen. Wayne Morse ( D. , Ore. ) in connection with President Eisenhower's cabinet selections in 1953 and President Kennedy's in 1961 . 

The best tag sequence is: start at nn bedz vbn in nn in nn-tl np np ( np , np ) in nn in nn-tl np$ nn nns in cd cc nn-tl np$ in cd . end 

The probability of the best tag sequence is: 5.4053474427923603e-95
Word 'Several' current best two-tag sequence: start ap
Word 'defendants' current best two-tag sequence: ap nns
Word 'in' current best two-tag sequence: nns in
Word 'the' current best two-tag sequence: i

Word 'includes' current best two-tag sequence: pps vbz
Word 'a' current best two-tag sequence: vbz at
Word 'raise' current best two-tag sequence: at nn
Word 'in' current best two-tag sequence: nn in
Word 'the' current best two-tag sequence: in at
Word 'county' current best two-tag sequence: at nn
Word 'minimum' current best two-tag sequence: nn nn
Word 'wage' current best two-tag sequence: jj nn
Word ',' current best two-tag sequence: nn ,
Word 'creation' current best two-tag sequence: , nn
Word 'of' current best two-tag sequence: nn in
Word 'several' current best two-tag sequence: in ap
Word 'new' current best two-tag sequence: ap jj
Word 'jobs' current best two-tag sequence: jj nns
Word 'at' current best two-tag sequence: nns in
Word 'the' current best two-tag sequence: in at
Word 'executive' current best two-tag sequence: at nn
Word 'level' current best two-tag sequence: jj nn
Word ',' current best two-tag sequence: nn ,
Word 'financing' current best two-tag sequence: , vbg
Word 'of

Word 'it' current best two-tag sequence: rb pps
Word 'spared' current best two-tag sequence: pps vbd
Word 'us' current best two-tag sequence: vbd ppo
Word 'from' current best two-tag sequence: ppo in
Word 'the' current best two-tag sequence: in at
Word 'usual' current best two-tag sequence: at jj
Word 'spate' current best two-tag sequence: jj nn
Word 'of' current best two-tag sequence: nn in
Word 'silly' current best two-tag sequence: in jj
Word 'resolutions' current best two-tag sequence: jj nns
Word 'which' current best two-tag sequence: nns wdt
Word 'in' current best two-tag sequence: wdt in
Word 'the' current best two-tag sequence: in at
Word 'past' current best two-tag sequence: at nn
Word 'have' current best two-tag sequence: nn hv
Word 'made' current best two-tag sequence: hv vbn
Word 'Georgia' current best two-tag sequence: vbn np
Word 'look' current best two-tag sequence: np vb
Word 'like' current best two-tag sequence: vb cs
Word 'anything' current best two-tag sequence: cs p

Word 'an' current best two-tag sequence: bez at
Word 'interesting' current best two-tag sequence: at jj
Word 'and' current best two-tag sequence: jj cc
Word 'exceptionally' current best two-tag sequence: cc ql
Word 'well-written' current best two-tag sequence: fw-uh-tl at
Word 'book' current best two-tag sequence: fw-uh-tl at
Word '.' current best two-tag sequence: fw-uh-tl at
The sentence was: On the whole this is an interesting and exceptionally well-written book . 

The best tag sequence is: start fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl at end 

The probability of the best tag sequence is: 0.0
Word 'A' current best two-tag sequence: start at
Word 'bunch' current best two-tag sequence: at nn
Word 'of' current best two-tag sequence: nn in
Word 'young' current best two-tag sequence: in jj
Word 'buckaroos' current best two-tag sequence: fw-uh-tl at
Word 'from' current best two-tag sequence: fw-uh-tl at
Word 'out' current best tw

...

Word 'this' current best two-tag sequence: bez* dt
Word 'nice' current best two-tag sequence: dt jj
Word '''' current best two-tag sequence: jj ''
The sentence was: But isn't this nice '' 

The best tag sequence is: start cc bez* dt jj '' end 

The probability of the best tag sequence is: 8.691449147611735e-18
Word 'Now' current best two-tag sequence: start rb
Word 'I' current best two-tag sequence: rb ppss
Word 'can' current best two-tag sequence: ppss md
Word 'only' current best two-tag sequence: md rb
Word 'write' current best two-tag sequence: rb vb
Word ',' current best two-tag sequence: vb ,
Word 'and' current best two-tag sequence: , cc
Word 'that' current best two-tag sequence: cc cs
Word 'only' current best two-tag sequence: cs rb
Word 'when' current best two-tag sequence: rb wrb
Word 'I' current best two-tag sequence: wrb ppss
Word 'get' current best two-tag sequence: ppss vb
Word 'out' current best two-tag sequence: vb rp
Word 'of' current best two-tag sequence: rp in
Word '

Word 'thinking' current best two-tag sequence: bedz vbg
Word 'about' current best two-tag sequence: vbg in
Word 'that' current best two-tag sequence: in dt
Word '.' current best two-tag sequence: dt .
The sentence was: I was thinking about that . 

The best tag sequence is: start ppss bedz vbg in dt . end 

The probability of the best tag sequence is: 4.045954056392396e-14
Word 'I' current best two-tag sequence: start ppss
Word 'thought' current best two-tag sequence: ppss vbd
Word 'I' current best two-tag sequence: vbd ppss
Word 'saw' current best two-tag sequence: ppss vbd
Word 'a' current best two-tag sequence: vbd at
Word 'faint' current best two-tag sequence: at jj
Word 'surge' current best two-tag sequence: jj nn
Word 'of' current best two-tag sequence: nn in
Word 'color' current best two-tag sequence: in nn
Word 'rise' current best two-tag sequence: nn nn
Word 'to' current best two-tag sequence: nn in
Word 'her' current best two-tag sequence: in pp$
Word 'neck' current best two-

Word 'this' current best two-tag sequence: vbd dt
Word 'expectation' current best two-tag sequence: dt nn
Word 'of' current best two-tag sequence: nn in
Word 'fun' current best two-tag sequence: in nn
Word 'and' current best two-tag sequence: nn cc
Word 'pleasure' current best two-tag sequence: cc nn
Word '.' current best two-tag sequence: nn .
The sentence was: The very faces of the people bore this expectation of fun and pleasure . 

The best tag sequence is: start at ap nns in at nns vbd dt nn in nn cc nn . end 

The probability of the best tag sequence is: 1.2589536404548978e-38
Word 'It' current best two-tag sequence: start pps
Word 'was' current best two-tag sequence: pps bedz
Word 'an' current best two-tag sequence: bedz at
Word 'automatic' current best two-tag sequence: at jj
Word 'phrase' current best two-tag sequence: jj nn
Word ';' current best two-tag sequence: nn .
Word ';' current best two-tag sequence: . .
The sentence was: It was an automatic phrase ; ; 

The best tag s

Word '?' current best two-tag sequence: . .
The sentence was: `` Is that you , Father ? ? 

The best tag sequence is: start `` bez cs ppss , nn-tl . . end 

The probability of the best tag sequence is: 8.345066033835073e-22
Word 'How' current best two-tag sequence: start wrb
Word 'did' current best two-tag sequence: wrb dod
Word 'one' current best two-tag sequence: dod pn
Word 'attach' current best two-tag sequence: pn vb
Word 'anything' current best two-tag sequence: vb pn
Word 'to' current best two-tag sequence: pn to
Word 'a' current best two-tag sequence: in at
Word 'stone' current best two-tag sequence: at nn
Word 'wall' current best two-tag sequence: nn nn
Word ',' current best two-tag sequence: nn ,
Word 'for' current best two-tag sequence: , in
Word 'that' current best two-tag sequence: in dt
Word 'matter' current best two-tag sequence: dt nn
Word '?' current best two-tag sequence: nn .
Word '?' current best two-tag sequence: . .
The sentence was: How did one attach anything to

Word 'are' current best two-tag sequence: dts ber
Word 'the' current best two-tag sequence: ber at
Word 'things' current best two-tag sequence: at nns
Word 'I' current best two-tag sequence: nns ppss
Word 'built' current best two-tag sequence: ppss vbd
Word 'the' current best two-tag sequence: vbd at
Word 'workshop' current best two-tag sequence: at nn
Word 'for' current best two-tag sequence: nn in
Word '''' current best two-tag sequence: in ''
Word ',' current best two-tag sequence: '' ,
Word 'protested' current best two-tag sequence: , vbd
Word 'Mr.' current best two-tag sequence: vbd np
Word 'Crombie' current best two-tag sequence: fw-uh-tl at
Word '.' current best two-tag sequence: fw-uh-tl at
The sentence was: `` But those are the things I built the workshop for '' , protested Mr. Crombie . 

The best tag sequence is: start fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl at end 

The p

In [244]:
Viterbi(['He', 'goes', 'to', 'asdas', 'school'])

Word 'He' current best two-tag sequence: start pps
Word 'goes' current best two-tag sequence: pps vbz
Word 'to' current best two-tag sequence: vbz to
Word 'asdas' current best two-tag sequence: fw-uh-tl at
Word 'school' current best two-tag sequence: fw-uh-tl at
fw-uh-tl    at 

fw-uh-tl    fw-uh-tl 

fw-uh-tl    fw-uh-tl 

fw-uh-tl    fw-uh-tl 

start    fw-uh-tl 

The sentence was: He goes to asdas school 

The best tag sequence is: start fw-uh-tl fw-uh-tl fw-uh-tl fw-uh-tl at end 

The probability of the best tag sequence is: 0.0


['start', 'fw-uh-tl', 'fw-uh-tl', 'fw-uh-tl', 'fw-uh-tl', 'at', 'end']